# Portfolio Update #2: Data Cleaning and Visualization

In this "tidy data" project, I will be loading in a messy dataset on the 2008 Olympic medalists and doing some cleaning and basic exploratory data analysis (EDA) on it. To run all of the code for yourself, make sure to download the "TidyData-Project" folder from my data science portfolio on GitHub (https://github.com/t-clark04/Clark-Data-Science-Portfolio/tree/main). The Olympic medalist data will be contained within the "data" folder, and all other packages and dependencies will be explained and loaded in along the way. Thank you for checking out my project!

First things first, we need to load in the .csv file as a pandas DataFrame and take a look at the first few rows of data to get of sense of what we're dealing with.

In [7]:
# Importing pandas so that we can create a pandas DataFrame from our .csv file
import pandas as pd

# Reading in the data using the read_csv() function in pandas and assigning
# the resulting dataframe to the variable df_messy
df_messy = pd.read_csv("data/olympics_08_medalists.csv")

# Using the .head() method to dispaly the first 10 rows of the DataFrame
df_messy.head(10)

,medalist_name,male_archery,female_archery,male_athletics,female_athletics,male_badminton,female_badminton,male_baseball,male_basketball,female_basketball,...,female_beach volleyball,male_volleyball,female_volleyball,male_water polo,female_water polo,male_weightlifting,female_weightlifting,male_freestyle wrestling,female_freestyle wrestling,male_greco-roman wrestling
0,Aaron Armstrong,NaN,NaN,gold,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Peirsol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abdullo Tangriev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abeer Abdelrahman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,bronze,NaN,NaN,NaN
4,Abhinav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Acer Nethercott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Adam Korol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Adam Kreek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adam Pine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Adam Wheeler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bronze


It looks like the first column of our dataset contains the name of each Olympic medalist (which will eventually need to be our identifier variable). The rest of the column names are Olympic sports, with each cell containing whether or not the given athlete earned an Olympic medal in that sport (specifying gold, silver, or bronze if true).

So, the first thing we should do here is use the .melt() method to pivot the DataFrame to long format. We'll set medalist_name as our identifier variable, convert the rest of the column names to their own variable called "Sport", and assign the re-formatted values to a variable called "Medal". Then, we'll take a look to see what we have after that.

In [11]:
# Using the .melt() method to convert our Olympic medal data from wide to long format
df_tidy = pd.melt(df_messy,
                  id_vars = "medalist_name",
                  value_vars = df_messy.columns[1:],
                  var_name = "Sport",
                  value_name = "Medal")

df_tidy

,medalist_name,Sport,Medal
0,Aaron Armstrong,male_archery,NaN
1,Aaron Peirsol,male_archery,NaN
2,Abdullo Tangriev,male_archery,NaN
3,Abeer Abdelrahman,male_archery,NaN
4,Abhinav,male_archery,NaN
...,...,...,...
131245,Éver Banega,male_greco-roman wrestling,NaN
131246,Ólafur Stefánsson,male_greco-roman wrestling,NaN
131247,Óscar Brayson,male_greco-roman wrestling,NaN
131248,Łukasz Pawłowski,male_greco-roman wrestling,NaN


We're still stuck with a bunch of NaN values in the medal column, since we now have every possible combination of athlete_name and sport in this dataset, even though each athlete only earned a medal in one sport.

So next, we'll drop all of the rows showing NaN in the medal column, since they don't really tell us anything.

In [13]:
# Using the .dropna() method to drop every row with missing data.
df_tidy = df_tidy.dropna()

df_tidy

,medalist_name,Sport,Medal
177,Bair Badënov,male_archery,bronze
676,Ilario Di Buò,male_archery,silver
682,Im Dong-hyun,male_archery,gold
760,Jiang Lin,male_archery,bronze
920,Lee Chang-hwan,male_archery,gold
...,...,...,...
130932,Steeve Guénot,male_greco-roman wrestling,gold
131159,Yannick Szczepaniak,male_greco-roman wrestling,bronze
131162,Yavor Yanakiev,male_greco-roman wrestling,bronze
131200,Yury Patrikeyev,male_greco-roman wrestling,bronze


That's more like it! But we can actually clean up this dataset even more by separating out the sport gender from the sport name, since they are currently both contained within the "Sport" column. We'll use the 